# Brooder and Finisher Optimizer for Turkey Logistics

## Objective and Prerequisites

A large turkey grower must transport young turkeys from barns called brooders to barns
called finishers. There are 10 brooders and 62 finishers within a ~50 mile radius of the
processing facility. Currently, the company transports birds from the brooder to the first
available (or one of a few available) finisher even if they must transport to the furthest finisher
from the brooder that is being emptied that week. Therefore, if a linear program (LP) could be
formulated seeking to minimize the distance traveled the company may benefit by reducing
transportation costs and bird mortality.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/bazylhorsey/livestock-logistic-optimizer). 

**Gurobi License** <br />
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MFG-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_food-manufacturing_2_COM_EVAL_GITHUB_&utm_term=food-manufacturing-problem&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MFG-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_food-manufacturing_2_ACADEMIC_EVAL_GITHUB_&utm_term=food-manufacturing-problem&utm_content=C_JPM) as an *academic user*.

---
## Problem Description

The objective function of the problem will be to minimize the miles driven between
brooders and finishers over one year
Before constraints can be written, a proper understanding of the turkey growing operation
must be acquired. The birds (called “poults” at this point in their life) are purchased and
delivered at one day old from nearby hatcheries and spend roughly the first 35 days of their lives
living in barns called brooders. A brooder can typically hold anywhere between 100,000 and
170,000 poults. After 35 days have elapsed and the poults have nearly outgrown the brooder barn
they are transported on trucks (approximately 3,000 at a time) to farms called finishers. A
finisher is comprised of two to five barns with each barn having a capacity of 11,000 birds. After
brooder barns are emptied they are cleaned and the bedding (of sawdust) is changed; this process
typically takes about 19 days. Once birds have been emptied into a finisher they grow for an
additional 105 days before being loading onto trucks and are taken to the plant. After a finisher is
emptied it is cleaned over a period of roughly 20 days and prepared for the next flock of birds.
Several years ago there was an international pandemic of avian flu which culled the birds
of many poultry operations in the region and around the world. Since that avian flu outbreak
poultry growers have enacted strict biosecurity measures to ensure the health and quality of their
birds. One such measure is to mandate that no birds of different age groups are mixed together.
For practical purposes, “same age” is taken to mean that they are born within one week apart.

There are a number of factors which might be relevant in the “real world” but cannot be
considered in this model due the difficulty involved in representing them mathematically. Some
of these factors include:

- Bird mortality: typically mortality is roughly 3% in brooders and 12% in finishers. Bird
mortality could be described as stochastic and is dependent on travel time, weather, bird
stress, and overall bird health

- Birds of the same age in different brooders: if birds of the same age are raised in different
brooders it is permissible for them to be placed at the same finisher

- Inexactness of capacity: while brooders and finishers have certain capacities it is unusual
for them to contain that exact amount of birds. For example, sometimes a finisher barn
may be slightly overfilled with 11,500 birds or underfilled with only 10,000 birds.
Additionally, sometimes brooders will contain fewer than their capacity allows due to
mortality or the hatchery slightly underdelivering on an order

- Inexactness of growing and cleaning times: often birds may stay in a brooder or finisher
for longer than 35 or 105 days. For example, if the processing facility is unable to keep
up with the production schedule (which was determine 1-2 years in advance) due to
insufficient labor, birds may end up staying in a finisher for an additional 30 days or
more. Additionally, a farmer may take shorter or longer than 19 or 20 days to clean their
barn after it is emptied

While important to consider, these points will not be factored into this particular model
formulation. However, should the company wish to run a full-scale model if this preliminary one
proves successful some of these may be wise to consider.


For the purposes of this model there are three main types of constraints and
considerations:
- Turkey life cycle
  - Brooders
    - Each brooder is occupied for 35 days prior to being emptied
    - Each brooder is cleaned over a period of 19 days after it is emptied
    - Therefore, it could be said that the average brooder is emptied (365/54) = 6.759 times per year
  - Finishers
    - Each finisher is occupied for 105 days prior to being emptied
    - Each finisher is cleaned over a period of 20 days after it is emptied
    - Therefore, it could be said that the average finisher receives (365/125) = 2.92 flocks of birds from brooders per year
- Farm and truck capacity
  - Brooders
    - Each brooder has a certain capacity of between 110,000 and 170,000
    - All brooders must be completely emptied after 35 days
  - Finishers
    - Each finisher may contain anywhere between 2 and 5 barns
    - Each barn has a capacity of 11,000 birds
    - Therefore, each finisher (farm) has a capacity of 22,000, 33,000, 44,000, or 55,000
    - The capacity of birds a finisher/ a group of finishers may receive from any given brooder over a year must be equal to or exceed the amount of birds that are actually sent to it
  - Trucks
    - Each truck can carry only 3,000 birds at a time
    - Therefore, the number of truckloads required to transport birds from brooders to finishers are 8, 11, 15, and 19 for finishers of capacity 22,000, 33,000, 44,000, 55,000 respectively (truckloads are rounded up because there can be no fraction of a load)
- Biosecurity
  - Birds may only be in the same brooder or finisher if they are born no more than a week apart
  - (For this scaled-down problem) Each finisher may be assigned only one brooder. i.e. even if birds in different brooders are of the same age they still cannot be grouped together in the same finisher

In [55]:
import pandas as pd

df = pd.read_csv("data/finisher.csv", nrows=29)
df


,Capacity,Distance Coefficient
0,55000,19
1,55000,19
2,55000,19
3,55000,19
4,55000,19
5,55000,19
6,55000,19
7,55000,19
8,55000,19
9,55000,19


---
## Model Formulation

### Sets and Indices

$f \in \text{Finishers}=\{\text{0..n\}}$: Serial set of finishers.

$b \in \text{Brooders}=\{\text{0..n\}}$: Serial set of brooders.


### Parameters

$\text{capacity}_f \in \mathbb{N}^+$: Capacity of finisher $f$.

$\text{capacity}_b \in \mathbb{N}^+$: Capacity of brooder $b$.

$\text{distance}_{b,f} \in \mathbb{R}^+$: Distance between brooder $b$ and finisher $f$ (in miles).

### Decision Variables

$\text{x}_{b, f} \in \{0,1\}$: 1 if finisher $f$ is to recieve birds from brooder $b$, 0 otherwise.


### Objective Function

- **Distance**: Minimize the total distance (in miles) to deliver brooder stock to finishers.

\begin{equation}
\text{Minimize} \quad Z = 2.994 \sum_{f \in \text{Finishers}}\sum_{b \in \text{Brooders}}(\text{distance}_{b,f}*\text{x}_{b, f})
\tag{0}
\end{equation}

### Constraints

<!-- - **Initial Balance:** The Tons of oil $o$ purchased in January and the ones previously stored should be equal to the Tons of said oil consumed and stored in that month.

\begin{equation}
\text{init store} + \text{buy}_{Jan,o} = \text{consume}_{Jan,o} + \text{store}_{Jan,o} \quad \forall o \in \text{Oils}
\tag{1}
\end{equation}

- **Balance:** The Tons of oil $o$ purchased in month $t$ and the ones previously stored should be equal to the Tons of said oil consumed and stored in that month.

\begin{equation}
\text{store}_{t-1,o} + \text{buy}_{t,o} = \text{consume}_{t,o} + \text{store}_{t,o} \quad \forall (t,o) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Oils}
\tag{2}
\end{equation}

- **Inventory Target**: The Tons of oil $o$ kept in inventory at the end of the planning horizon should hit the target.

\begin{equation}
\text{store}_{Jun,o} = \text{target_store} \quad \forall o \in \text{Oils}
\tag{3}
\end{equation}

- **Refinement Capacity**: Total Tons of oil $o$ consumed in month $t$ cannot exceed the refinement capacity.

\begin{equation}
\sum_{o \in V}\text{consume}_{t,o} \leq \text{veg_cap} \quad \forall t \in \text{Months}
\tag{4.1}
\end{equation}

\begin{equation}
\sum_{o \in N}\text{consume}_{t,o} \leq \text{oil_cap} \quad \forall t \in \text{Months}
\tag{4.2}
\end{equation}

- **Hardness**: The hardness value of the food produced in month $t$ should be within tolerances.

\begin{equation}
\text{min_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \text{hardness}_o*\text{consume}_{t,o} \leq \text{max_hardness}*\text{produce}_t \quad \forall t \in \text{Months}
\tag{5}
\end{equation}

- **Mass Conservation**: Total Tons of oil consumed in month $t$ should be equal to the Tons of the food produced in that month.

\begin{equation}
\sum_{o \in \text{Oils}}\text{consume}_{t,o} = \text{produce}_t \quad \forall t \in \text{Months}
\tag{6}
\end{equation}

- **Consumption Range**: Oil $o$ can be consumed in month $t$ if we decide to use it in that month, and the Tons consumed should be between 20 and the refinement capacity for its type. 

\begin{equation}
\text{min_consume}*\text{use}_{t,o} \leq \text{consume}_{t,o} \leq \text{veg_cap}*\text{use}_{t,o} \quad \forall (t,o) \in V \times \text{Months}
\tag{7.1}
\end{equation}

\begin{equation}
\text{min_consume}*\text{use}_{t,o} \leq \text{consume}_{t,o} \leq \text{oil_cap}*\text{use}_{t,o} \quad \forall (t,o) \in N \times \text{Months}
\tag{7.2}
\end{equation}

- **Recipe**: The maximum number of oils used in month $t$ must be three.

\begin{equation}
\sum_{o \in \text{Oils}}\text{use}_{t,o} \leq \text{max_ingredients} \quad \forall t \in \text{Months}
\tag{8}
\end{equation}

- **If-then Constraint**: If oils VEG1 or VEG2 are used in month $t$, then OIL3 must be used in that month.

\begin{equation}
\text{use}_{t,\text{VEG1}} \leq \text{use}_{t,\text{OIL3}} \quad \forall t \in \text{Months}
\tag{9.1}
\end{equation}

\begin{equation}
\text{use}_{t,\text{VEG2}} \leq \text{use}_{t,\text{OIL3}} \quad \forall t \in \text{Months}
\tag{9.2}
\end{equation} -->

---
## Python Implementation
We import the Gurobi Python Module and other Python libraries.

In [ ]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9

## Input Data
We define all the input data of the model.

In [ ]:
# Parameters

import pandas as pd

distances_frame = pd.read_csv("data/distance.csv", index_col=None)
brooders_frame = pd.read_csv("data/brooder.csv")
finishers_frame = pd.read_csv("data/finisher.csv")


FINISHERS = [*range(0, len(df.axes[0]) + 1)]
BROODERS = [*range(0, len(df.axes[1]) + 1)]

brooder_capacity = []
for index, row in brooders_frame.iterrows():
    brooder_capacity.append(row[0])
    
finisher_capacity = []
finisher_distance_k = []
for index, row in finishers_frame.iterrows():
    finisher_capacity.append(row[0])
    finisher_distance_k.append(row[1])

# example
# distance = distances_frame.to_dict()

# print(distance
# TODO: { (f,b):d, (f,b):d, ... }

distance = {(0,0):13}

# for i, finisher in distances_frame.iterrows():
#     print(dict(finisher))

        
        

BROODER_OUTPUT = 6.715
FINISHER_INPUT = 2.944


## Model Deployment

Explain


In [ ]:
# model goes here

Explain what constraints do

In [ ]:
# constraints go here

More on constraints?

In [ ]:
# constraints go here

More on constraints?

In [ ]:
# constraints go here

More on constraints

The objective is to minimize overall distance traveled.

In [ ]:
# Objective Function


Next, we start the optimization and Gurobi finds the optimal solution.

In [ ]:
minimize_distance.optimize()

---
## Analysis

Analysis and thoughts

### Travel Plan
Defines the amount of miles driven if the model is used.

In [ ]:
# Dataframe (table 2 in excel with pandas)

### Chosen Route Plan

Defines which route a brooder is allowed to deliver to within the constraints.

In [ ]:
# Dataframe (table 3 in excel with pandas)

Note: If you want to write your solution to a file, rather than print it to the terminal, you can use the model.write() command. An example implementation is:

`minimize_distance.write("food-manufacture-2-output.sol")`

---
## References

Copyright © Griffin Wilson and Bazyl Horsey